In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os,random, math
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

# from dataprep.eda import plot, plot_correlation, plot_missing

import plotly 
import plotly.express as px
# from plotly import tools, subplots
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# from plotly.offline import init_notebook_mode, iplot
# import plotly.graph_objs as go
# import plotly.express as px

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

## 법정동 정리

In [7]:
bjd = pd.read_csv("bupjeongdong.csv")

In [8]:
bjd

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
46287,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재
46288,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재
46289,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재
46290,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재


https://www.code.go.kr/stdcode/regCodeL.do

In [12]:
l = []
for c in bjd.법정동코드:
    l.append(len(str(c)))
pd.DataFrame(l).value_counts()

10    46292
dtype: int64

In [18]:
str(bjd.법정동코드[0])[:5]

'11000'

In [ ]:
list(map(lambda x:str.split()))

In [21]:
bjd["시군구"] = list(map(lambda x: str(x)[:5] , bjd.법정동코드))
bjd["법정동"] = list(map(lambda x: str(x)[5:] , bjd.법정동코드))

In [22]:
bjd

,법정동코드,법정동명,폐지여부,시군구,법정동
0,1100000000,서울특별시,존재,11000,00000
1,1111000000,서울특별시 종로구,존재,11110,00000
2,1111010100,서울특별시 종로구 청운동,존재,11110,10100
3,1111010200,서울특별시 종로구 신교동,존재,11110,10200
4,1111010300,서울특별시 종로구 궁정동,존재,11110,10300
...,...,...,...,...,...
46287,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재,50130,32022
46288,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재,50130,32023
46289,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재,50130,32024
46290,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재,50130,32025


In [35]:
address = list(map(lambda x: x.split(" ") , bjd.법정동명))

In [40]:
pd.DataFrame(list(map(lambda x: x[0] , address))).value_counts()

경상북도       9224
경기도        6522
경상남도       6093
충청남도       5267
전라남도       4821
전라북도       3350
충청북도       3147
강원도        2839
서울특별시      1112
부산직할시       454
대구광역시       368
제주도         357
인천광역시       344
대구직할시       334
부산광역시       297
광주직할시       272
울산광역시       248
광주광역시       246
인천직할시       223
대전직할시       220
제주특별자치도     216
대전광역시       184
세종특별자치시     154
dtype: int64

In [47]:
bjd["시군구1"] = list(map(lambda x: x[0] , address))
bjd["시군구2"] = list(map(lambda x: x[1] if len(x)>1 else '' , address))
bjd["법정동1"] = list(map(lambda x: x[2] if len(x)>2 else '' , address))
bjd["법정동2"] = list(map(lambda x: x[3] if len(x)>3 else '' , address))

In [48]:
bjd

,법정동코드,법정동명,폐지여부,시군구,법정동,시군구1,시군구2,법정동1,법정동2
0,1100000000,서울특별시,존재,11000,00000,서울특별시,,,
1,1111000000,서울특별시 종로구,존재,11110,00000,서울특별시,종로구,,
2,1111010100,서울특별시 종로구 청운동,존재,11110,10100,서울특별시,종로구,청운동,
3,1111010200,서울특별시 종로구 신교동,존재,11110,10200,서울특별시,종로구,신교동,
4,1111010300,서울특별시 종로구 궁정동,존재,11110,10300,서울특별시,종로구,궁정동,
...,...,...,...,...,...,...,...,...,...
46287,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재,50130,32022,제주특별자치도,서귀포시,표선면,하천리
46288,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재,50130,32023,제주특별자치도,서귀포시,표선면,성읍리
46289,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재,50130,32024,제주특별자치도,서귀포시,표선면,가시리
46290,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재,50130,32025,제주특별자치도,서귀포시,표선면,세화리


## 법정동 test set 과 merge

In [55]:
exdata = pd.read_csv("Daesamanlap/External/external_data.csv")

In [51]:
test = pd.read_excel("test_imputed.xlsx")

In [62]:
ad = pd.read_excel("address.xlsx")
ad.columns = ['NO', '입력주소', '우편번호', '도로명주소', '기타주소']
ad = ad.iloc[1:,]

In [52]:
test.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,지하철역,버스정류장,단지내주차면수,단지명,도로명주소,임대보증금,임대료
0,C1072,754,아파트,경기도,국민임대,39.7900,116,14,H,0.0000,2,683,광명역세권 휴먼시아2단지,경기도 광명시 서원로 25,"22,830,000.0000","189,840.0000"
1,C1072,754,아파트,경기도,국민임대,46.8100,30,14,A,0.0000,2,683,광명역세권 휴먼시아2단지,경기도 광명시 서원로 25,"36,048,000.0000","249,930.0000"
2,C1072,754,아파트,경기도,국민임대,46.9000,112,14,H,0.0000,2,683,광명역세권 휴먼시아2단지,경기도 광명시 서원로 25,"36,048,000.0000","249,930.0000"
3,C1072,754,아파트,경기도,국민임대,46.9000,120,14,H,0.0000,2,683,광명역세권 휴먼시아2단지,경기도 광명시 서원로 25,"36,048,000.0000","249,930.0000"
4,C1072,754,아파트,경기도,국민임대,51.4600,60,14,H,0.0000,2,683,광명역세권 휴먼시아2단지,경기도 광명시 서원로 25,"43,497,000.0000","296,780.0000"


In [63]:
ad.head()

,NO,입력주소,우편번호,도로명주소,기타주소
1,1.0000,서울특별시 종로구 종로31가길 25-5,03129,서울특별시 종로구 종로31가길 25-5 (연지동),NaN
2,2.0000,서울특별시 종로구 종로56길 23,03114,서울특별시 종로구 종로56길 23 (숭인동),NaN
3,3.0000,서울특별시 종로구 종로63라길 29,03113,서울특별시 종로구 종로63라길 29 (숭인동),NaN
4,4.0000,서울특별시 종로구 종로 19,03157,서울특별시 종로구 종로 19 (종로1가),NaN
5,5.0000,서울특별시 종로구 종로31길 46-30,03129,서울특별시 종로구 종로31길 46-30 (연지동),NaN


In [53]:
test.isnull().sum()

단지코드         0
총세대수         0
임대건물구분       0
지역           0
공급유형         0
전용면적         0
전용면적별세대수     0
공가수          0
자격유형         2
지하철역        42
버스정류장        0
단지내주차면수      0
단지명          0
도로명주소        0
임대보증금        0
임대료          0
dtype: int64

In [65]:
exdata[exdata.주택유형명 == "아파트"]

,기관명,광역시도코드,광역시도명,시군구코드,시군구명,단지명,도로명주소,세대수,공급유형명,형명,공급전용면적,공급공용면적,주택유형명,주차수,기본임대보증금,기본월임대료,기본전환보증금
86,SH공사,11,서울특별시,110,종로구,경희궁자이(돈의문1구역),서울특별시 종로구 송월길 130,308,50년임대,39.0561,39.0561,19.6373,아파트,501,53140000,179900,0
87,SH공사,11,서울특별시,110,종로구,경희궁자이(돈의문1구역),서울특별시 종로구 송월길 130,308,50년임대,42.1289,42.1289,19.9740,아파트,501,56610000,191600,0
88,SH공사,11,서울특별시,110,종로구,경희궁자이(돈의문1구역),서울특별시 종로구 송월길 130,308,50년임대,44.0260,44.0260,20.8796,아파트,501,59160000,200300,0
89,SH공사,11,서울특별시,110,종로구,경희궁자이(돈의문1구역),서울특별시 종로구 송월길 130,308,50년임대,54.1632,54.1632,26.1670,아파트,501,73070000,247300,0
114,SH공사,11,서울특별시,110,종로구,무악현대(무악1),서울특별시 종로구 통일로 246-11,550,50년임대,15,32.9300,19.8400,아파트,205,10600000,149200,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97459,LH제주,50,제주특별자치도,130,서귀포시,보목신환,제주특별자치도 서귀포시 마소물로 3,2,국민임대,55,55.0740,15.7876,아파트,0,20156000,111960,10000000
97460,LH제주,50,제주특별자치도,130,서귀포시,제주서귀포혁신도시 A1블록,제주특별자치도 서귀포시 서호남로 92-15,119,10년임대,59,59.9800,19.7740,아파트,0,27431000,482790,69000000
97461,LH제주,50,제주특별자치도,130,서귀포시,남원에덴아파트,제주특별자치도 서귀포시 남원읍 남원광지로 5,6,국민임대,40,40.0000,7.0000,아파트,0,8865000,60260,0
97462,LH제주,50,제주특별자치도,130,서귀포시,남원에덴아파트,제주특별자치도 서귀포시 남원읍 남원광지로 5,6,국민임대,60,60.0000,7.0000,아파트,0,15957000,132380,14000000


In [70]:
test.도로명주소[0]+"\n"

'경기도 광명시 서원로 25\n'

In [71]:
f = open("도로명주소.txt", 'w')
for a in list(test.도로명주소):
    data = a+"\n"
    f.write(data)
f.close()

https://cafe.naver.com/excelmaster/152667

In [3]:
pop1 = pd.read_excel("/Users/seungji/Desktop/Dacon/Daesamanlap/External2/202106주민등록인구및세대현황_월간.xlsx")
pop2 = pd.read_excel("/Users/seungji/Desktop/Dacon/Daesamanlap/External2/202106연령별인구현황_월간.xlsx")

In [13]:
tr = pd.read_csv("/Users/seungji/Desktop/Dacon/Daesamanlap/ProcessedData/train_processed.csv")
tr

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철역,버스정류장,단지내주차면수,등록차량수,단지명,도로명주소
0,C2483,900,아파트,경상북도,국민임대,39.7200,134,38.0000,A,"15,667,000.0000","103,680.0000",0.0000,3.0000,"1,425.0000","1,015.0000",포항장량휴먼시아1단지,경상북도 포항시 북구 장량중앙로 17
1,C2483,900,아파트,경상북도,국민임대,39.7200,15,38.0000,A,"15,667,000.0000","103,680.0000",0.0000,3.0000,"1,425.0000","1,015.0000",포항장량휴먼시아1단지,경상북도 포항시 북구 장량중앙로 17
2,C2483,900,아파트,경상북도,국민임대,51.9300,385,38.0000,A,"27,304,000.0000","184,330.0000",0.0000,3.0000,"1,425.0000","1,015.0000",포항장량휴먼시아1단지,경상북도 포항시 북구 장량중앙로 17
3,C2483,900,아파트,경상북도,국민임대,51.9300,15,38.0000,A,"27,304,000.0000","184,330.0000",0.0000,3.0000,"1,425.0000","1,015.0000",포항장량휴먼시아1단지,경상북도 포항시 북구 장량중앙로 17
4,C2483,900,아파트,경상북도,국민임대,51.9300,41,38.0000,A,"27,304,000.0000","184,330.0000",0.0000,3.0000,"1,425.0000","1,015.0000",포항장량휴먼시아1단지,경상북도 포항시 북구 장량중앙로 17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2571,481,상가,경상남도,임대상가,31.8400,1,5.0000,D,"5,580,000.0000","71,900.0000",0.0000,1.0000,65.0000,62.0000,통영도남주공아파트,경상남도 통영시 발개로 153
2948,C2571,481,상가,경상남도,임대상가,31.8400,1,5.0000,D,"5,580,000.0000","71,900.0000",0.0000,1.0000,65.0000,62.0000,통영도남주공아파트,경상남도 통영시 발개로 153
2949,C2571,481,상가,경상남도,임대상가,31.8400,1,5.0000,D,"5,580,000.0000","71,900.0000",0.0000,1.0000,65.0000,62.0000,통영도남주공아파트,경상남도 통영시 발개로 153
2950,C2571,481,상가,경상남도,임대상가,31.8400,1,5.0000,D,"5,580,000.0000","71,900.0000",0.0000,1.0000,65.0000,62.0000,통영도남주공아파트,경상남도 통영시 발개로 153


In [4]:
pop1.head()

,행정기관코드,행정기관,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
0,1100000000,서울특별시,"9,565,990","4,405,638",2.1700,"4,649,442","4,916,548",0.9500
1,1111000000,서울특별시 종로구,"146,029","73,525",1.9900,"70,985","75,044",0.9500
2,1111051500,서울특별시 종로구 청운효자동,"12,024","5,242",2.2900,"5,580","6,444",0.8700
3,1111053000,서울특별시 종로구 사직동,"9,371","4,674",2.0000,"4,225","5,146",0.8200
4,1111054000,서울특별시 종로구 삼청동,"2,498","1,248",2.0000,"1,191","1,307",0.9100


In [6]:
pop2.head(2)

,행정기관코드,행정기관,총인구수,총연령구간인구수,총0~9세,총10~19세,총20~29세,총30~39세,총40~49세,총50~59세,총60~69세,총70~79세,총80~89세,총90~99세,총100세이상,남 인구수,남연령구간인구수,남0~9세,남10~19세,남20~29세,...,남40~49세,남50~59세,남60~69세,남70~79세,남80~89세,남90~99세,남100세 이상,여 인구수,여연령구간인구수,여0~9세,여10~19세,여20~29세,여30~39세,여40~49세,여50~59세,여60~69세,여70~79세,여80~89세,여90~99세,여100세 이상
0,1100000000,서울특별시,"9,565,990","9,565,990","602,526","766,572","1,443,208","1,452,328","1,501,071","1,506,232","1,269,804","696,049","284,923","40,808","2,469","4,649,442","4,649,442","309,269","392,686","696,930",...,"743,762","741,675","599,745","317,147","111,573","10,322",619,"4,916,548","4,916,548","293,257","373,886","746,278","726,614","757,309","764,557","670,059","378,902","173,350","30,486","1,850"
1,1111000000,서울특별시 종로구,"146,029","146,029","7,275","10,684","23,147","19,595","21,433","24,939","20,192","11,791","5,985",936,52,"70,985","70,985","3,651","5,311","11,255",...,"10,450","12,579","10,055","5,224","2,413",288,16,"75,044","75,044","3,624","5,373","11,892","9,852","10,983","12,360","10,137","6,567","3,572",648,36


In [11]:
print(bjd.shape)
print(pop1.shape)
print(pop2.shape)

(46292, 3)
(3846, 8)
(3847, 41)


In [16]:
bjd1 = bjd.merge(pop1, left_on = ["법정동코드"], right_on =["행정기관코드"],how = "inner")
bjd1

,법정동코드,법정동명,폐지여부,행정기관코드,행정기관,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
0,1100000000,서울특별시,존재,1100000000,서울특별시,"9,565,990","4,405,638",2.1700,"4,649,442","4,916,548",0.9500
1,1111000000,서울특별시 종로구,존재,1111000000,서울특별시 종로구,"146,029","73,525",1.9900,"70,985","75,044",0.9500
2,1114000000,서울특별시 중구,존재,1114000000,서울특별시 중구,"123,016","62,897",1.9600,"60,019","62,997",0.9500
3,1117000000,서울특별시 용산구,존재,1117000000,서울특별시 용산구,"226,378","111,650",2.0300,"108,948","117,430",0.9300
4,1120000000,서울특별시 성동구,존재,1120000000,서울특별시 성동구,"289,162","134,542",2.1500,"141,042","148,120",0.9500
...,...,...,...,...,...,...,...,...,...,...,...
1678,5013025000,제주특별자치도 서귀포시 대정읍,존재,5013025000,제주특별자치도 서귀포시 대정읍,"21,959","10,523",2.0900,"10,687","11,272",0.9500
1679,5013025300,제주특별자치도 서귀포시 남원읍,존재,5013025300,제주특별자치도 서귀포시 남원읍,"18,717","8,677",2.1600,"9,668","9,049",1.0700
1680,5013025900,제주특별자치도 서귀포시 성산읍,존재,5013025900,제주특별자치도 서귀포시 성산읍,"15,478","8,165",1.9000,"8,105","7,373",1.1000
1681,5013031000,제주특별자치도 서귀포시 안덕면,존재,5013031000,제주특별자치도 서귀포시 안덕면,"12,205","6,288",1.9400,"6,306","5,899",1.0700


In [17]:
bjd2 = bjd.merge(pop1, left_on = ["법정동명"], right_on =["행정기관"],how = "inner")
bjd2

,법정동코드,법정동명,폐지여부,행정기관코드,행정기관,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
0,1111011500,서울특별시 종로구 사직동,존재,1111053000,서울특별시 종로구 사직동,"9,371","4,674",2.0000,"4,225","5,146",0.8200
1,1111014000,서울특별시 종로구 삼청동,존재,1111054000,서울특별시 종로구 삼청동,"2,498","1,248",2.0000,"1,191","1,307",0.9100
2,1111014600,서울특별시 종로구 가회동,존재,1111060000,서울특별시 종로구 가회동,"4,210","2,030",2.0700,"1,944","2,266",0.8600
3,1111016500,서울특별시 종로구 이화동,존재,1111064000,서울특별시 종로구 이화동,"7,347","4,289",1.7100,"3,472","3,875",0.9000
4,1111016900,서울특별시 종로구 혜화동,존재,1111065000,서울특별시 종로구 혜화동,"16,489","9,194",1.7900,"7,949","8,540",0.9300
...,...,...,...,...,...,...,...,...,...,...,...
2229,5013025000,제주특별자치도 서귀포시 대정읍,존재,5013025000,제주특별자치도 서귀포시 대정읍,"21,959","10,523",2.0900,"10,687","11,272",0.9500
2230,5013025300,제주특별자치도 서귀포시 남원읍,존재,5013025300,제주특별자치도 서귀포시 남원읍,"18,717","8,677",2.1600,"9,668","9,049",1.0700
2231,5013025900,제주특별자치도 서귀포시 성산읍,존재,5013025900,제주특별자치도 서귀포시 성산읍,"15,478","8,165",1.9000,"8,105","7,373",1.1000
2232,5013031000,제주특별자치도 서귀포시 안덕면,존재,5013031000,제주특별자치도 서귀포시 안덕면,"12,205","6,288",1.9400,"6,306","5,899",1.0700


In [23]:
len(set(bjd1.법정동코드) | set(bjd2.법정동코드))

2523

In [40]:
len(set(bjd2.법정동코드)- set(bjd1.법정동코드))

840

In [37]:
pop1[pop1.행정기관 == "제주특별자치도 제주시 애월읍 용흥리"]

,행정기관코드,행정기관,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율


In [35]:
bjd.법정동명[46100:46200]

46100        제주특별자치도 제주시 외도일동
46101        제주특별자치도 제주시 외도이동
46102        제주특별자치도 제주시 이호일동
46103        제주특별자치도 제주시 이호이동
46104        제주특별자치도 제주시 도두일동
46105        제주특별자치도 제주시 도두이동
46106         제주특별자치도 제주시 도남동
46107        제주특별자치도 제주시 도련일동
46108        제주특별자치도 제주시 도련이동
46109         제주특별자치도 제주시 용강동
46110         제주특별자치도 제주시 회천동
46111         제주특별자치도 제주시 오등동
46112         제주특별자치도 제주시 월평동
46113         제주특별자치도 제주시 영평동
46114          제주특별자치도 제주시 연동
46115         제주특별자치도 제주시 도평동
46116         제주특별자치도 제주시 해안동
46117         제주특별자치도 제주시 내도동
46118         제주특별자치도 제주시 한림읍
46119     제주특별자치도 제주시 한림읍 귀덕리
46120     제주특별자치도 제주시 한림읍 수원리
46121     제주특별자치도 제주시 한림읍 대림리
46122     제주특별자치도 제주시 한림읍 한림리
46123     제주특별자치도 제주시 한림읍 금악리
46124     제주특별자치도 제주시 한림읍 동명리
46125     제주특별자치도 제주시 한림읍 명월리
46126     제주특별자치도 제주시 한림읍 옹포리
46127     제주특별자치도 제주시 한림읍 상대리
46128     제주특별자치도 제주시 한림읍 상명리
46129     제주특별자치도 제주시 한림읍 협재리
46130     제주특별자치도 제주시 한림읍 월령리
46131     제주특별자치도 제주시 한림읍 금능리
46132     제주특별자치도 제주시 한림읍 월림리
46133     